In [2]:
import libm2k
import numpy as np
import math, sys
from tqdm import tqdm

available_sample_rates= [750, 7500, 75000, 750000, 7500000, 75000000]
max_rate = available_sample_rates[-1] # last sample rate = max rate
min_nr_of_points=10
max_buffer_size = 500000
uri = "ip:192.168.3.2"

def get_best_ratio(ratio, thresh):
    max_it=max_buffer_size/ratio
    best_ratio=ratio
    best_fract=1

    for i in range(1,int(max_it)):
        new_ratio = i*ratio
        (new_fract, integral) = math.modf(new_ratio)
        if new_fract < best_fract and int(new_ratio) % 4 == 0:
            best_fract = new_fract
            best_ratio = new_ratio
            
        if best_ratio > 200000:
            return 999999999,0
            
            #print(best_ratio, best_fract)
        if best_fract < thresh:
            break

    return int(best_ratio),best_fract


def get_samples_count(rate, freq):
    ratio = rate/freq
    
    print("rat", ratio)
    
    if ratio<min_nr_of_points and rate < max_rate:
        return 0
    if ratio<2:
        return 0

    ratio,fract = get_best_ratio(ratio, 0.0002)
    # ratio = number of periods in buffer
    # fract = what is left over - error

    size=int(ratio)
    while size & 0x03:
        size=size<<1
    while size < 1024:
        size=size<<1
    return size

def get_optimal_sample_rate(freq):
    for rate in available_sample_rates:
        buf_size = get_samples_count(rate,freq)
        if buf_size:
            return rate

def sine_buffer_generator(channel, sample_rate, freq, ampl, size = None):

    buffer = []

    nr_of_samples = size if size is not None else get_samples_count(sample_rate, freq)
    samples_per_period = sample_rate / freq

    print("sample_rate:",sample_rate)
    print("number_of_samples",nr_of_samples)
    print("samples_per_period",samples_per_period)
        
    for i in range(nr_of_samples):
        buffer.append(ampl * (math.sin(((i)/samples_per_period) * 2*math.pi) ))

    return sample_rate, buffer




In [21]:
min_freq = 7040000 # 3570000 #7040000
max_freq = min_freq + 200

#min_freq = 14097000
#max_freq = 14097200

offset = 1.4648
rate = 75000000
thresh = 0.0001

out = []

for poss in tqdm(np.arange(min_freq, max_freq, 1)):
    waves = [get_best_ratio(rate / (poss + offset * i), thresh)[0] for i in range(4)]
    out.append((poss, max(waves), waves))
    
drr = {}
for freq, max_waves, waves in sorted(out, key=lambda o: o[1]):
    waves = [max([x * i for i in range(1000) if x * i <= max_waves]) for x in waves]
    print(freq, max_waves, waves)
    drr[freq] = waves

100%|██████████| 200/200 [00:08<00:00, 25.00it/s]

7040053 95848 [95848, 85184, 95816, 76640]
7040000 999999999 [7492500, 999999999, 999999999, 999999999]
7040001 999999999 [999999999, 999999999, 20327652, 999999999]
7040002 999999999 [999999999, 999999999, 999999999, 999999999]
7040003 999999999 [999999999, 26745228, 999999999, 999999999]
7040004 999999999 [999999999, 999999999, 132619248, 42773184]
7040005 999999999 [999999999, 999999999, 191420388, 191388420]
7040006 999999999 [999999999, 37430532, 999999999, 122944932]
7040007 999999999 [119784096, 999999999, 999999999, 63068868]
7040008 999999999 [999999999, 999999999, 999999999, 999999999]
7040009 999999999 [999999999, 187088724, 999999999, 999999999]
7040010 999999999 [999999999, 999999999, 27788184, 999999999]
7040011 999999999 [999999999, 999999999, 54501444, 127148724]
7040012 999999999 [999999999, 999999999, 5342652, 77989932]
7040013 999999999 [999999999, 999999999, 57694248, 45934020]
7040014 999999999 [999999999, 999999999, 56619324, 65154780]
7040015 999999999 [999999999

In [22]:
base_freq = 7040053 #7040028 # 14097071 #7020050
sizes = drr[base_freq] # [73060, 50816, 56100, 40216] # [56948, 70504, 82432, 85140]

waves = [sine_buffer_generator(0, rate, base_freq + offset*i , 10, sizes[i])[1] for i in range(4)]

sample_rate: 75000000
number_of_samples 95848
samples_per_period 10.653328888291039
sample_rate: 75000000
number_of_samples 85184
samples_per_period 10.653326671689417
sample_rate: 75000000
number_of_samples 95816
samples_per_period 10.653324455088715
sample_rate: 75000000
number_of_samples 76640
samples_per_period 10.653322238488938


In [23]:
ctx=libm2k.m2kOpen()

ctx.calibrateDAC()

siggen=ctx.getAnalogOut()
siggen.setSampleRate(0, rate)
siggen.enableChannel(0, True)
siggen.setCyclic(True)
siggen.push(0, waves[0])


In [24]:
bits = [3,3,0,0,2,0,0,0,1,0,0,2,3,1,3,0,2,0,1,2,2,1,2,3,1,1,1,2,2,2,2,0,0,0,1,0,0,3,2,1,2,0,0,2,0,0,1,0,1,1,0,2,1,1,0,1,0,0,2,1,3,2,1,2,2,2,0,3,1,2,3,2,3,2,3,2,3,0,2,3,2,0,3,2,1,3,2,2,0,3,3,0,3,0,1,2,2,0,3,2,0,2,2,0,1,0,2,3,2,2,3,1,1,0,3,3,2,2,1,1,2,1,2,2,2,1,1,3,2,2,2,0,2,1,0,1,2,0,3,1,0,2,2,0,2,0,0,3,3,2,3,2,3,1,2,0,0,3,1,2,2,2]


import time
import datetime

while True:
    time_delay = 0.683
    i = 0
    last_time = 0

    siggen.push([waves[bits[i]]])

    while not (datetime.datetime.now().minute % 2 == 0 and datetime.datetime.now().second == 1):
        pass


    start_time = time.time()

    while True:
        if (time.time() - start_time) >= time_delay * i:
            i += 1
            if i == len(bits): break
            siggen.push([waves[bits[i]]])

            print("{}/{} - {:.04f}".format(i, len(bits), time.time() - last_time))
            last_time = time.time()



1/162 - 1625989921.0624
2/162 - 0.6918
3/162 - 0.6799
4/162 - 0.6799
5/162 - 0.6801
6/162 - 0.6898
7/162 - 0.6802
8/162 - 0.6787
9/162 - 0.6917
10/162 - 0.6799
11/162 - 0.6791
12/162 - 0.6885
13/162 - 0.6805
14/162 - 0.6793
15/162 - 0.6816
16/162 - 0.6903
17/162 - 0.6798
18/162 - 0.6787
19/162 - 0.6914
20/162 - 0.6796
21/162 - 0.6793
22/162 - 0.6911
23/162 - 0.6774
24/162 - 0.6810
25/162 - 0.6799
26/162 - 0.6901
27/162 - 0.6813
28/162 - 0.6797
29/162 - 0.6900
30/162 - 0.6804
31/162 - 0.6796
32/162 - 0.6804
33/162 - 0.6900
34/162 - 0.6785
35/162 - 0.6811
36/162 - 0.6899
37/162 - 0.6780
38/162 - 0.6819
39/162 - 0.6892
40/162 - 0.6810
41/162 - 0.6888
42/162 - 0.6813
43/162 - 0.6791
44/162 - 0.6809
45/162 - 0.6792
46/162 - 0.6893
47/162 - 0.6710
48/162 - 0.6886
49/162 - 0.6901
50/162 - 0.6717
51/162 - 0.6890
52/162 - 0.6890
53/162 - 0.6696
54/162 - 0.6915
55/162 - 0.6788
56/162 - 0.6910
57/162 - 0.6798
58/162 - 0.6804
59/162 - 0.6887
60/162 - 0.6789
61/162 - 0.6820
62/162 - 0.6790
63/162 -

KeyboardInterrupt: 

In [12]:
for i in range(99999999): siggen.push([waves[i % 4]])

KeyboardInterrupt: 

In [ ]:
import time

In [ ]:
waves[0][-10:]